# Team 16 Toyota SmartHome ML Training

## Setting up dependencies

#### Check CUDA Version
To check if CUDA is available, perform the following:<br>
1) Go to device manager
2) Expand Display adapters
3) Go to https://en.wikipedia.org/wiki/CUDA#GPUs_supported <br>(https://developer.nvidia.com/cuda-gpus has an outdated list e.g. GTX 1050 Ti is not included)
4) If exists, Open Command prompt and execute "nvidia-smi" or "nvcc --version"<br>
5) Install pytorch according the CUDA version displayed.<br>
<b>CUDA 10.2</b>: No longer available for windows<br>
<b>CUDA 11.3</b>: conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch<br>
<b>CUDA 11.6</b>: conda install pytorch torchvision torchaudio cudatoolkit=11.6 -c pytorch -c conda-forge<br>
<b>CPU</b>: conda install pytorch torchvision torchaudio cpuonly -c pytorch<br>

#### Additional Dependencies

Setup Virtual Environment
Run in Terminal:
- virtualenv .venv
- pip install -r requirements.txt

<!-- #### Importing Libraries -->

In [2]:
from ipywidgets import widgets, Button
from IPython.display import display, HTML, Video
from ipyfilechooser import FileChooser
import zipfile as zipfile
import os
import io
import base64
import json


### Hide code cells

In [16]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

<!-- ### Go to: -->

<!-- <a href="#TSU">Toyota TSU section</a><br>
<a href="#STEP">Nvidia STEP section</a><br>
<a href='#MS-TCT'>MS-TCT section</a> -->

## Data Exploration

### File Upload

The following cell is to upload the file in the respective folder.

In [17]:
# Creates FileChooser and setting default directory (https://pypi.org/project/ipyfilechooser/)
fc = FileChooser('/Users')
# Restrict navigation to /Users
fc.sandbox_path = '/Users'
# Change hidden files
fc.show_hidden = False
# Customize dir icon
# fc.dir_icon = '/'
# fc.dir_icon_append = True
def submit_upload(b):
    fn = fc.selected_filename
    identified_filetype = ""
    if fn.endswith('.zip'):
        fn = fc.selected_filename.replace('.zip', '')
        with zipfile.ZipFile(fc.selected_path+"\\"+fc.selected_filename, 'r') as zipObj:
            zip_content = zipObj.namelist()
            for zip_content_item in zip_content:
                if zip_content_item.endswith('.mp4'):
                    zipObj.extract(zip_content_item, 'Data_Storage\\Videos')
                    if identified_filetype != "":
                        identified_filetype += ", Video"
                    else:
                        identified_filetype += "Video"
                elif zip_content_item.endswith('.csv') or zip_content_item.endswith('.xlsx') or zip_content_item.endswith('.txt'):
                    zipObj.extract(zip_content_item, 'Data_Storage\\Dataset')
                    if identified_filetype != "":
                        identified_filetype += ", Dataset"
                    else:
                        identified_filetype += "Dataset"
                elif zip_content_item.endswith('.file'):
                    zipObj.extract(zip_content_item, 'Data_Storage\\Pretrained_models')
                    if identified_filetype != "":
                        identified_filetype += ", Pretrained Model"
                    else:
                        identified_filetype += "Pretrained"
                elif zip_content_item.endswith('.json') or zip_content_item.endswith('.vtt'):
                    zipObj.extract(zip_content_item, 'Data_Storage\\Captions')
                    if identified_filetype != "":
                        identified_filetype += ", Caption"
                    else:
                        identified_filetype += "Caption"
                else:
                    zipObj.extract(zip_content_item, 'Data_Storage')
                    if identified_filetype != "":
                        identified_filetype += ", Others"
                    else:
                        identified_filetype += "Others"
    else:
        identified_filetype = ""
        if fn.endswith('.mp4'):  
            shutil.copyfile(fc.selected_path+"\\"+fc.selected_filename, 'Data_Storage\\Videos\\'+fc.selected_filename)
            identified_filetype += ", Videos" 
        elif fn.endswith('.csv') or fn.endswith('.xlsx') or fn.endswith('.txt'):
            shutil.copyfile(fc.selected_path+"\\"+fc.selected_filename, 'Data_Storage\\Dataset\\'+fc.selected_filename)
            identified_filetype += "Dataset"    
        elif fn.endswith('.file'):
            shutil.copyfile(fc.selected_path+"\\"+fc.selected_filename, 'Data_Storage\\Pretrained_Models\\'+fc.selected_filename)
            identified_filetype += "Pretrained_Model"
        elif fn.endswith('.json') or fn.endswith('.vtt'):
            shutil.copyfile(fc.selected_path+"\\"+fc.selected_filename, 'Data_Storage\\Captions\\'+fc.selected_filename)
            identified_filetype += ", Captions"
        else:
            shutil.copyfile(fc.selected_path+"\\"+fc.selected_filename, 'Data_Storage\\'+fc.selected_filename)
            identified_filetype += ", Others"
    with output:
        print("File type identified to be/include: " + identified_filetype)

# Set input file types
# submit_upload() function in "function.py" sorts input for video, csv/xslx(Dataset)
confirmBtn = widgets.Button(
    description='Confirm Upload',
    tooltip='Confirm',
)
output = widgets.Output()
display(fc)
display(confirmBtn, output)
confirmBtn.on_click(submit_upload)


FileChooser(path='C:\Users', filename='', title='', show_hidden=False, select_desc='Select', change_desc='Chan…

Button(description='Confirm Upload', style=ButtonStyle(), tooltip='Confirm')

Output()

### Adding pipeline
This cell will be used to allow the user to select the pipeline.

In [18]:
ppl = ['TSU', 'STEP' , 'MS-TCT']
ppDropdown = widgets.Dropdown(
    options=ppl,
    value=ppl[0],
    description='Select Pipeline:')

display(ppDropdown)

Dropdown(description='Select Pipeline:', options=('TSU', 'STEP', 'MS-TCT'), value='TSU')

### Displaying video with generated captions<br>
Here, we will look at selecting a video to generate the captions using the pre-trained model before displaying the video with the captions.

In [19]:

def getVideoList():
    video_folder = os.listdir('Data_Storage/Videos')
    list_of_video = {}
    for video in video_folder:
        # remove hidden file especially for macOS
        if not video.endswith('.mp4'):
            list_of_video[video] = video
    return list_of_video


# read action and frame object from file
def readCaptionFile(filename,videoName):
    # Opening JSON file
    f = open(filename)

    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # Iterating through the json
    # Closing file
    f.close()
    return data[videoName]["actions"]

# convert frame to time
def convertFrameToTime(frame):
    seconds = int(frame/25)
    minutes = "00"
    if seconds >= 60:
        minutes = str(seconds // 60)
        seconds = seconds % 60
    if len(minutes) == 1:
        minutes = "0" + minutes
    seconds = str(seconds)
    #may need handle hour
    if len(seconds) == 1:
        seconds = "0" + seconds 
    return (minutes + ":" + seconds + ".000")

# read reference text from txt file
def readReferenceFile(refFile):
    referenceDict = {}
    with open(refFile) as f:
        lines = f.readlines()
    for i in lines:
        x = i.split()
        referenceDict[str(x[0])] = x[1]
    return referenceDict

# create caption file
def formatCaptionFile(captionList, reference, captionPath):
    start = "WEBVTT\n\n"
    captions = []
    for i in captionList:
        text = reference[str(i[0])]
        lines = convertFrameToTime(i[1]) + " --> " + convertFrameToTime(i[2]) + "\n" + text + "\n\n"
        captions.append(lines)
    f = open(captionPath, "w")
    f.write(start)
    f.writelines(captions)
    f.close()

output_test = widgets.Output()
# ========== FileChooser ==========
# Create and display a FileChooser widget
fc = FileChooser('./Data_Storage/Videos')
display(fc)

def play_video(f):
    fn = fc.selected_filename
    fn = fn.split('.')
    src = "./Data_Storage/Videos/" + fc.selected_filename
    video = io.open(src, 'r+b').read()
    encoded = base64.b64encode(video)
    #location of reference are place at root
    ref = readReferenceFile('./Data_Storage/all_labels.txt')
    # may need change the caption path to dynamic
    
    caption_location = './Data_Storage/Captions/' + fn[0] + ".vtt"
    # model result file should be some directory, here using root 
    captionList = readCaptionFile('./Data_Storage/smarthome_CS_51.json', fn[0])
    formatCaptionFile(captionList, ref, caption_location)
    display(HTML(data='''<video width="650" height="360" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        <track kind="captions" src={1} srclang="en" label="English" default>
        </video>'''.format(encoded.decode('ascii'), caption_location)))
        

fc.sandbox_path = './Data_Storage/Videos'
fc.show_hidden = False
fc.show_only_dirs = False
fc.title = '<b>Select input video</b>'
fc.register_callback(play_video)

FileChooser(path='C:\Users\danie\Desktop\ict3104-team16-2022-New-UI-Interface\Data_Storage\Videos', filename='…

## Inference

### Feature Extraction

The following codes will allow the users to select the model and video that will be used for inference.

In [20]:
mtpList = ['Pretrained_Models', 'Trained_Models']
mtp = widgets.Dropdown( #Model Type Picker
    options = mtpList,
    default = 0,
    description = 'Model Type',
)

def display_inference_model(f):
    rs = fc.selected_path+fc.selected_filename
    with selectedptmodel:
        print(rs + "has been selected.")

selectedptmodel = widgets.Output()
ptmodel = FileChooser('./Data_Storage/'+ mtp.value)
ptmodel.sandbox_path = './Data_Storage/'+ mtp.value
ptmodel.show_hidden = False
ptmodel.filter_pattern = ['*']
ptmodel.show_only_dirs = False
ptmodel.title = '<b>Select Inference Model</b>'
ptmodel.register_callback(display_inference_model)


def getVideoList():
    video_folder = os.listdir('./Data_Storage/Videos')
    list_of_video = {}
    for video in video_folder:
        # remove hidden file especially for macOS
        if video.endswith('.mp4'):
            list_of_video[video] = video
    return list_of_video


display(ptmodel)
display(selectedptmodel)

# run = f"python test.py -dataset TSU -mode rgb -split_setting CS -model PDAN -train False -num_channel 512 -lr 0.0002 -kernelsize 3 -APtype map -batch_size 1 -comp_info TSU_CS_RGB_PDAN -load_model {model_src}"
# % {run}



FileChooser(path='C:\Users\danie\Desktop\ict3104-team16-2022-New-UI-Interface\Data_Storage\Pretrained_Models',…

Output()

In [21]:
vl = getVideoList()
vlDropdown = widgets.Dropdown(
    options=vl,
    description='Select Video:')


confirmBtn = widgets.Button(
    description='Confirm',
    tooltip='Confirm',
)


display(vlDropdown)
display(confirmBtn)

Dropdown(description='Select Video:', options={'P02T01C06.mp4': 'P02T01C06.mp4', 'P02T01C07.mp4': 'P02T01C07.m…

Button(description='Confirm', style=ButtonStyle(), tooltip='Confirm')

## Training a model

### The following codes will allow the user to train a ML model.

#### Set values for training

In [22]:
#Imports
import ipywidgets as widgets
from IPython.display import clear_output

trainBool = False


batchsize_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=4,
    step=1,
    description='Batch size:',
    continuous_update=False
)

epoch_slider = widgets.IntSlider(
    value=1000,
    min=0,
    max=1000,
    step=10,
    description='Epochs:',
    continuous_update=False
)

run_training_model = widgets.Button(
    description='Start Training',
    tooltip='Run',
)


pretrained_model_folder = os.listdir('./Data_Storage/Pretrained_Models')

list_of_ptModels = {}
for model in pretrained_model_folder:
    # remove hidden file especially for macOS
    if not model.startswith('.'):
        list_of_ptModels[model] = model

evaluation_dropdown = widgets.Dropdown(
    options = list_of_ptModels,
    description = 'evaluation',
)

model_src = './Data_Storage/Pretrained_Models/' + evaluation_dropdown.value

def start_training():
    clear_output(wait=False)
    print("\n Starting the process...")
    run = f"python test.py -dataset TSU -mode rgb -split_setting CS -model PDAN -train False -num_channel 512 -lr 0.0002 -kernelsize 3 -APtype map -batch_size 1 -comp_info TSU_CS_RGB_PDAN -load_model {model_src}"
    !{run}


display(epoch_slider)
display(batchsize_slider)    
display(run_training_model)

run_training_model.on_click(start_training)


IntSlider(value=1000, continuous_update=False, description='Epochs:', max=1000, step=10)

IntSlider(value=2, continuous_update=False, description='Batch size:', max=4, min=1)

Button(description='Start Training', style=ButtonStyle(), tooltip='Run')

### Renaming the Trained Models

In [28]:
selectedModelLoc = ""

fcModelRN = FileChooser('./Data_Storage/Captions')
fcModelRN.sandbox_path = './Data_Storage/Captions'
fcModelRN.show_hidden = False
fcModelRN.filter_pattern = ['*']
fcModelRN.show_only_dirs = False
fcModelRN.title = '<b>Select File to rename</b>'


changeName_btn = widgets.Button(
    description='Start Training',
    tooltip='Run',
)

# callback function
def getName(chooser):
    global selectedModelLoc 
    selectedModelLoc= fcModelRN.selected_path+fcModelRN.selected_filename

display(fcModelRN)
fcModelRN.register_callback(getName)


FileChooser(path='C:\Users\danie\Desktop\ict3104-team16-2022-New-UI-Interface\Data_Storage\Captions', filename…

In [29]:
modelNN = widgets.Text(
        placeholder='New Model Name',
        disabled=False
    )
saveButton = widgets.Button(
        description='Save',
        disabled=False,
    )

# callback function
def changeName(chooser):
    newModelLoc = fcModelRN.selected_path + modelNN.value
    if os.path.exists(newModelLoc):
         print("File name has already been used!")
    else:
        print(selectedModelLoc)
        print(newModelLoc)
        os.rename( selectedModelLoc, newModelLoc)

saveButton.on_click(changeName)
if selectedModelLoc != "":
    display(modelNN)
    display(saveButton)

Text(value='', placeholder='New Model Name')

Button(description='Save', style=ButtonStyle())